In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import seaborn as sns


# GETHERING DATA

In [2]:
patients = pd.read_csv('DATA/patients.csv')
adversal_reaction = pd.read_csv('DATA/adverse_reactions.csv')

In [3]:
#split contact colum into phone and email
patient_co = patients.copy()

In [4]:
patient_co.head()

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi
0,1,female,Zoe,Wellish,576 Brown Bear Drive,Rancho California,California,92390.0,United States,951-719-9170ZoeWellish@superrito.com,7/10/1976,121.7,66,19.6
1,2,female,Pamela,Hill,2370 University Hill Road,Armstrong,Illinois,61812.0,United States,PamelaSHill@cuvox.de+1 (217) 569-3204,4/3/1967,118.8,66,19.2
2,3,male,Jae,Debord,1493 Poling Farm Road,York,Nebraska,68467.0,United States,402-363-6804JaeMDebord@gustr.com,2/19/1980,177.8,71,24.8
3,4,male,Liêm,Phan,2335 Webster Street,Woodbridge,NJ,7095.0,United States,PhanBaLiem@jourrapide.com+1 (732) 636-8246,7/26/1951,220.9,70,31.7
4,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,AL,36303.0,United States,334-515-7487TimNeudorf@cuvox.de,2/18/1928,192.3,27,26.1


In [5]:
patient_co['patient_number'] =patient_co.contact.str.extract('((?:\+\d{1,2}\s)?\(?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4})', expand=True)

In [6]:
patient_co['patient_number'].head()

0         951-719-9170
1    +1 (217) 569-3204
2         402-363-6804
3    +1 (732) 636-8246
4         334-515-7487
Name: patient_number, dtype: object

In [7]:
patient_co['patient_email'] = patient_co.contact.str.extract('([a-zA-Z][a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+[a-zA-Z])', expand=True)

In [8]:
patient_co.head()

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi,patient_number,patient_email
0,1,female,Zoe,Wellish,576 Brown Bear Drive,Rancho California,California,92390.0,United States,951-719-9170ZoeWellish@superrito.com,7/10/1976,121.7,66,19.6,951-719-9170,ZoeWellish@superrito.com
1,2,female,Pamela,Hill,2370 University Hill Road,Armstrong,Illinois,61812.0,United States,PamelaSHill@cuvox.de+1 (217) 569-3204,4/3/1967,118.8,66,19.2,+1 (217) 569-3204,PamelaSHill@cuvox.de
2,3,male,Jae,Debord,1493 Poling Farm Road,York,Nebraska,68467.0,United States,402-363-6804JaeMDebord@gustr.com,2/19/1980,177.8,71,24.8,402-363-6804,JaeMDebord@gustr.com
3,4,male,Liêm,Phan,2335 Webster Street,Woodbridge,NJ,7095.0,United States,PhanBaLiem@jourrapide.com+1 (732) 636-8246,7/26/1951,220.9,70,31.7,+1 (732) 636-8246,PhanBaLiem@jourrapide.com
4,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,AL,36303.0,United States,334-515-7487TimNeudorf@cuvox.de,2/18/1928,192.3,27,26.1,334-515-7487,TimNeudorf@cuvox.de


In [9]:
patient_co =patient_co.drop('contact' ,axis=1)

In [10]:
treatment = pd.read_csv('DATA/treatments.csv')
treatment.head()

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
0,veronika,jindrová,41u - 48u,-,7.63,7.20,NaN
1,elliot,richardson,-,40u - 45u,7.56,7.09,0.97
2,yukitaka,takenaka,-,39u - 36u,7.68,7.25,NaN
3,skye,gormanston,33u - 36u,-,7.97,7.62,0.35
4,alissa,montez,-,33u - 29u,7.78,7.46,0.32


In [11]:
treatment_co = treatment.copy()
treatment_co =pd.melt(treatment_co, id_vars=['given_name', 'surname', 'hba1c_start', 'hba1c_end', 'hba1c_change'],
                           var_name='treatment', value_name='dose')
treatment_co['start_dose'] ,treatment_co['end_dose'] = treatment_co.dose.str.split(' - ',1).str

In [12]:
treatment_co =treatment_co.drop('dose' ,axis=1)

In [13]:
treatment_co[treatment_co.given_name == 'veronika']

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,treatment,start_dose,end_dose
0,veronika,jindrová,7.63,7.2,NaN,auralin,41u,48u
280,veronika,jindrová,7.63,7.2,NaN,novodra,-,NaN


In [14]:
treatment_co = pd.merge(treatment_co, adversal_reaction, on=['given_name', 'surname'], how='left')

In [15]:
len(treatment_co.columns)

9

In [16]:
treatment_co.columns.str.upper

<bound method _noarg_wrapper.<locals>.wrapper of <pandas.core.strings.StringMethods object at 0x0000008A3BE62C88>>

In [17]:
treatment_co.columns = map(str.lower, treatment_co.columns)

In [18]:
treatment_co.head()

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,treatment,start_dose,end_dose,adverse_reaction
0,veronika,jindrová,7.63,7.20,NaN,auralin,41u,48u,NaN
1,elliot,richardson,7.56,7.09,0.97,auralin,-,NaN,hypoglycemia
2,yukitaka,takenaka,7.68,7.25,NaN,auralin,-,NaN,NaN
3,skye,gormanston,7.97,7.62,0.35,auralin,33u,36u,NaN
4,alissa,montez,7.78,7.46,0.32,auralin,-,NaN,NaN


In [19]:
treatment_co['given_name'] =treatment_co['given_name'].str.lower()

In [20]:
treatment_co.head()

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,treatment,start_dose,end_dose,adverse_reaction
0,veronika,jindrová,7.63,7.20,NaN,auralin,41u,48u,NaN
1,elliot,richardson,7.56,7.09,0.97,auralin,-,NaN,hypoglycemia
2,yukitaka,takenaka,7.68,7.25,NaN,auralin,-,NaN,NaN
3,skye,gormanston,7.97,7.62,0.35,auralin,33u,36u,NaN
4,alissa,montez,7.78,7.46,0.32,auralin,-,NaN,NaN


In [21]:
treatment_co['hba1c_change'] =treatment_co.hba1c_start -treatment_co.hba1c_end

In [22]:
treatment_co.head()

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,treatment,start_dose,end_dose,adverse_reaction
0,veronika,jindrová,7.63,7.20,0.43,auralin,41u,48u,NaN
1,elliot,richardson,7.56,7.09,0.47,auralin,-,NaN,hypoglycemia
2,yukitaka,takenaka,7.68,7.25,0.43,auralin,-,NaN,NaN
3,skye,gormanston,7.97,7.62,0.35,auralin,33u,36u,NaN
4,alissa,montez,7.78,7.46,0.32,auralin,-,NaN,NaN


In [23]:
patient_co.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 15 columns):
patient_id        503 non-null int64
assigned_sex      503 non-null object
given_name        503 non-null object
surname           503 non-null object
address           491 non-null object
city              491 non-null object
state             491 non-null object
zip_code          491 non-null float64
country           491 non-null object
birthdate         503 non-null object
weight            503 non-null float64
height            503 non-null int64
bmi               503 non-null float64
patient_number    491 non-null object
patient_email     491 non-null object
dtypes: float64(3), int64(2), object(10)
memory usage: 59.0+ KB


In [24]:
patient_co.zip_code.head()

0    92390.0
1    61812.0
2    68467.0
3     7095.0
4    36303.0
Name: zip_code, dtype: float64

In [25]:
patient_co.zip_code =patient_co.zip_code.astype(str).str[:-2].str.pad(5,fillchar='0')

In [26]:
patient_co.zip_code.head()

0    92390
1    61812
2    68467
3    07095
4    36303
Name: zip_code, dtype: object

In [27]:
patient_co.zip_code.value_counts()
patient_co.zip_code = patient_co.zip_code.replace('0000n',np.nan)

In [28]:
patient_co.zip_code.value_counts()


12345    6
30303    4
10004    4
98109    3
15205    3
11590    3
11530    3
60148    3
35203    3
01730    3
70112    3
10011    3
90017    3
94108    2
94612    2
02081    2
40507    2
60605    2
97205    2
33323    2
10001    2
13221    2
92103    2
73102    2
92101    2
12771    2
07087    2
34471    2
95814    2
19108    2
        ..
78401    1
33301    1
62040    1
97321    1
72210    1
78212    1
74106    1
94509    1
86341    1
77591    1
83234    1
30346    1
28412    1
92801    1
08060    1
41858    1
18436    1
47957    1
33179    1
27055    1
15215    1
94070    1
87109    1
71201    1
55410    1
95204    1
52240    1
95060    1
02210    1
10570    1
Name: zip_code, Length: 430, dtype: int64

In [29]:
treatment_co = treatment_co[treatment_co['start_dose'] != '-']
treatment_co.start_dose = treatment_co.start_dose.str.strip('u').astype(int)
treatment_co

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,treatment,start_dose,end_dose,adverse_reaction
0,veronika,jindrová,7.63,7.20,0.43,auralin,41,48u,NaN
3,skye,gormanston,7.97,7.62,0.35,auralin,33,36u,NaN
6,sophia,haugen,7.65,7.27,0.38,auralin,37,42u,NaN
7,eddie,archer,7.89,7.55,0.34,auralin,31,38u,NaN
9,asia,woźniak,7.76,7.37,0.39,auralin,30,36u,NaN
10,joseph,day,7.70,7.19,0.51,auralin,29,36u,hypoglycemia
12,roxanne,andreyeva,9.54,9.14,0.40,auralin,29,38u,NaN
14,simone,baumgaertner,7.74,7.30,0.44,auralin,27,37u,NaN
15,enco,žibrik,7.78,7.34,0.44,auralin,55,68u,NaN
16,camilla,zaitseva,7.53,7.13,0.40,auralin,28,37u,NaN


In [30]:
treatment_co.end_dose = treatment_co.end_dose.str.strip('u').astype(int)
treatment_co

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,treatment,start_dose,end_dose,adverse_reaction
0,veronika,jindrová,7.63,7.20,0.43,auralin,41,48,NaN
3,skye,gormanston,7.97,7.62,0.35,auralin,33,36,NaN
6,sophia,haugen,7.65,7.27,0.38,auralin,37,42,NaN
7,eddie,archer,7.89,7.55,0.34,auralin,31,38,NaN
9,asia,woźniak,7.76,7.37,0.39,auralin,30,36,NaN
10,joseph,day,7.70,7.19,0.51,auralin,29,36,hypoglycemia
12,roxanne,andreyeva,9.54,9.14,0.40,auralin,29,38,NaN
14,simone,baumgaertner,7.74,7.30,0.44,auralin,27,37,NaN
15,enco,žibrik,7.78,7.34,0.44,auralin,55,68,NaN
16,camilla,zaitseva,7.53,7.13,0.40,auralin,28,37,NaN
